**Movie recommender**



Using GPU

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


Import libraries and datasets

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from google.colab import files
import io
import re

In [ ]:
uploaded = files.upload()

Saving movies.csv to movies (3).csv
Saving ratings_samp.csv to ratings_samp (3).csv
Saving tags.csv to tags (3).csv


In [ ]:
movie_df = pd.read_csv(io.BytesIO(uploaded['movies.csv']))
rating_df = pd.read_csv(io.BytesIO(uploaded['ratings_samp.csv']))
tag_df = pd.read_csv(io.BytesIO(uploaded['tags.csv']))

In [ ]:
rating_df1= rating_df.sample(frac=0.0005, replace=False, random_state=1)

In [ ]:
rating_df1.head()

,userId,movieId,rating,timestamp
120587,4829,5956,4.0,1277725632
171554,63546,3699,3.0,967476732
5506,101486,265,5.0,840120883
38370,134328,2761,4.0,1183260936
36930,16249,2610,4.0,945044465


In [ ]:
user_movies_data = rating_df1.pivot(index = "movieId", columns = "userId", values = "rating").fillna(0)

In [ ]:
user_movies_data

userId,256,1560,3179,3268,4352,4829,4937,5868,6593,6886,7159,8263,9562,10781,11987,12270,12793,15882,16249,17313,17602,18166,18409,18526,18706,20127,20785,20916,22071,23447,29663,29714,33767,34637,36097,37807,38448,42556,43704,43756,...,71093,73382,74788,74848,75581,77999,78773,79367,80554,80693,82418,82517,87340,89416,99352,101440,101486,101708,103448,104187,105054,107260,110435,111575,113513,113553,119531,119913,121535,122267,124644,125903,126524,126651,128050,134328,136660,137261,137373,137824
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64614,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64839,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
print(movie_df.head())
print(rating_df.head())
print(tag_df.head())

   movieId  ...                                           metadata
0        1  ...   Watched computer_animation Disney_animated_fe...
1        2  ...   time_travel adapted_from:book board_game chil...
2        6  ...   realistic_action Al_Pacino Michael_Mann Rober...
3       14  ...   biopic character_based_on_real_person:Richard...
4       18  ...   Quentin_Tarantino multiple_storylines Antonio...

[5 rows x 5 columns]
   userId  movieId  rating   timestamp
0  123425      529     4.0   840651205
1  108287      947     5.0   938960563
2  109381     4179     4.0   986858118
3   64404      909     4.5  1149562221
4  137309     1747     2.5  1109328180
   userId  movieId            tag   timestamp
0      18     4141    Mark_Waters  1240597180
1      65      208      dark_hero  1368150078
2      65      353      dark_hero  1368150079
3      65      521  noir_thriller  1368149983
4      65      592      dark_hero  1368150078


In [ ]:
tag_df.tag = tag_df["tag"].str.replace(" ", "_")

Join table Tag and Movie

In [ ]:
new_df = pd.merge(movie_df[["movieId"]], tag_df[["movieId", "tag"]], on = "movieId", how = "outer")
uniqnames = new_df.movieId.unique()
newdata = []                 # data list for output dataframe
for u in uniqnames:          # for each unique name
    subdf = new_df[new_df.movieId == u] # get rows with this unique name
    s = " "
    for i in subdf['tag']:
        s += str(i) + " "      # join all info cells for that name
    newdata.append([u, s[:-1]])
newdf = pd.DataFrame(data=newdata, columns=['movieId','tag'])

In [ ]:
newdf.head()

,movieId,tag
0,1,Watched computer_animation Disney_animated_fe...
1,2,time_travel adapted_from:book board_game chil...
2,6,realistic_action Al_Pacino Michael_Mann Rober...
3,14,biopic character_based_on_real_person:Richard...
4,18,Quentin_Tarantino multiple_storylines Antonio...


In [ ]:
movie_df['metadata'] = newdf['tag'] 
movie_df.head()

,movieId,movieId_x,movieId_y,title,metadata
0,1,1,1,Toy Story (1995),Watched computer_animation Disney_animated_fe...
1,2,2,2,Jumanji (1995),time_travel adapted_from:book board_game chil...
2,6,6,6,Heat (1995),realistic_action Al_Pacino Michael_Mann Rober...
3,14,14,14,Nixon (1995),biopic character_based_on_real_person:Richard...
4,18,18,18,Four Rooms (1995),Quentin_Tarantino multiple_storylines Antonio...


In [ ]:
movie_df = pd.merge(pd.DataFrame(user_movies_data.index), movie_df[["movieId", "title", "metadata"]], left_on = user_movies_data.index, right_on = "movieId", how = "left")

Content filtering 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(movie_df['metadata'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), index = movie_df.movieId.tolist())

In [ ]:
# if the table is too big, the system will consider it to have NaN or Inf because most values are zeros
tfidf_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2885,2886,2887,2888,2889,2890,2891,2892,2893,2894,2895,2896,2897,2898,2899,2900,2901,2902,2903,2904,2905,2906,2907,2908,2909,2910,2911,2912,2913,2914,2915,2916,2917,2918,2919,2920,2921,2922,2923,2924
6,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.017415,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.127558,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.127558,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.127558,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
47,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.011252,0.005626,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
150,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58295,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.163716,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64614,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64839,0.0,0.0,0.0,0.0,0.0,0.03314,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.019292,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.115755,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65261,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.045137,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0

In [ ]:

np.isinf(tfidf_df).any().sum()

0

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components = 2924)
latent_matrix = svd.fit_transform(tfidf_df)

In [ ]:
n=2925
latent_matrix_1_df = pd.DataFrame(latent_matrix[:,0:n], index = movie_df["title"].tolist())
latent_matrix_1_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98
Heat (1995),0.108740,0.042292,-0.051662,-0.020320,0.003485,0.046618,0.017967,-0.040759,0.004789,0.050408,-0.067154,0.102943,0.125879,-0.017547,-0.128849,0.080977,-0.022538,0.074626,-0.006782,-0.217673,-0.030754,0.010259,0.015965,0.051164,0.131303,0.032958,0.125162,0.067986,-0.013763,0.143572,0.262501,0.317094,-0.113697,0.119784,0.051887,-0.059615,-0.150465,-0.079046,0.247375,-0.025228,...,0.063320,-0.131945,0.001254,0.017585,-0.033615,-0.159695,-0.033571,-0.051749,-0.011416,0.049477,-0.003054,0.044746,0.033472,0.013163,-0.003678,-0.059570,-0.033209,-0.019998,-0.004056,0.006714,-0.022586,0.047469,-0.002292,-0.038997,-0.021404,-0.012513,0.029190,0.022028,0.011636,-0.026048,0.008754,-0.037233,0.017297,-0.020040,-0.005737,0.001960,-0.003024,0.008030,-0.009464,-0.005983
Powder (1995),0.015705,-0.002470,0.002066,0.003314,0.000237,0.015757,-0.004165,0.000114,-0.005938,-0.011935,-0.018763,0.031306,0.019226,-0.004209,-0.025051,-0.037643,0.008823,0.001317,0.042298,0.015842,0.031154,0.079568,0.050742,-0.061681,-0.037910,-0.047599,0.008278,0.063386,-0.005198,0.048419,-0.028288,-0.047144,0.054179,0.173173,0.324214,-0.164248,-0.083483,0.458085,0.301548,0.248817,...,0.126631,0.002429,-0.100689,-0.108455,0.013486,0.027378,-0.014918,0.004907,-0.018713,-0.009100,-0.023974,0.034442,0.009536,0.027533,0.010818,-0.024741,0.023421,0.009929,0.010592,0.014350,0.006961,-0.012793,-0.007327,0.005913,-0.003480,-0.011123,0.006950,-0.000189,-0.000930,0.001038,0.009543,-0.010144,0.001585,-0.000772,-0.010096,0.007003,0.007139,0.003093,-0.012664,-0.000028
Carrington (1995),0.012669,0.006861,0.002347,0.006953,0.005928,0.021781,0.004506,0.016043,-0.000860,-0.009404,-0.008807,0.000349,-0.016264,-0.025320,-0.022600,0.039857,-0.005295,-0.053633,0.061894,0.002154,-0.065407,0.082562,-0.090863,0.041336,0.044796,-0.012897,0.104561,-0.068503,-0.050302,0.063470,0.001055,-0.021649,0.161173,-0.222936,-0.065730,-0.195004,0.414370,0.132971,0.161235,0.098773,...,0.121065,-0.098418,-0.054071,0.041942,-0.088788,0.023273,-0.048571,0.004136,0.086039,-0.066194,0.038830,-0.006118,0.007212,-0.001307,0.016896,0.012674,0.002119,-0.005818,0.010056,0.011184,-0.005381,0.002904,0.008100,0.011394,-0.009104,-0.001981,-0.005811,0.002241,0.002512,0.004475,0.000330,0.000846,-0.000561,0.006397,-0.001927,0.001410,0.000165,-0.000062,0.001393,-0.001412
Seven (a.k.a. Se7en) (1995),0.298620,0.408771,-0.150110,-0.353396,-0.119387,-0.063576,-0.007439,-0.173968,0.059452,0.019025,0.011247,-0.098525,0.124014,-0.117524,-0.035714,0.005062,-0.056531,0.041057,0.061431,0.183127,-0.061149,0.021041,-0.028599,0.015383,0.000101,0.081508,-0.072614,-0.042251,-0.069049,-0.141864,0.020479,-0.034223,-0.058199,0.010953,-0.008785,-0.056410,-0.030367,0.025482,-0.014505,0.001092,...,-0.011044,0.022315,0.040310,-0.056909,0.010477,0.087505,-0.012430,-0.003880,-0.024635,0.081618,0.043479,0.033509,0.168468,-0.105969,0.105229,0.081341,-0.040676,-0.056927,0.046607,-0.003755,0.061169,-0.214722,0.161981,0.189914,-0.129753,-0.138371,0.298406,0.073922,0.039441,0.002427,-0.012121,-0.123165,0.098867,-0.046489,-0.001005,0.035292,0.034320,-0.034461,0.158726,-0.003848
Apollo 13 (1995),0.107733,0.028857,0.019417,0.070076,0.051807,0.329285,-0.073526,-0.130308,-0.117404,-0.008137,0.112045,-0.079595,-0.102387,0.131625,0.036124,0.266577,-0.047935,0.438427,0.100254,0.002445,0.017992,-0.154035,0.101889,-0.117748,0.037546,0.165343,0.206842,0.139467,-0.138880,0.167233,0.062122,-0.042868,0.043364,-0.032249,-0.036089,-0.129489,0.004372,0.001812,-0.080625,-0.009575,...,-0.014514,-0.027427,0.107710,-0.156241,0.012458,0.048353,0.087595,0.074300,0.025025,-0.045881,-0.054763,0.051372,-0.040699,0.089317,0.063902,0.068169,0.018943,0.049704,-0.078328,0.090765,-0.009487,0.017349,0.074893,0.004990,0.031612,-0.032587,-0.119040,

Collaborate filtering

In [ ]:
id_title = pd.merge(pd.DataFrame(user_movies_data.index), movie_df[["movieId", "title"]], left_on = user_movies_data.index, right_on = "movieId", how = "left")

In [ ]:
id_title = id_title[["movieId", "title"]]
id_title

,movieId,title
0,6,Heat (1995)
1,24,Powder (1995)
2,35,Carrington (1995)
3,47,Seven (a.k.a. Se7en) (1995)
4,150,Apollo 13 (1995)
...,...,...
94,58295,"Bank Job, The (2008)"
95,64614,Gran Torino (2008)
96,64839,"Wrestler, The (2008)"
97,65261,Ponyo (Gake no ue no Ponyo) (2008)


In [ ]:
from sklearn.decomposition import TruncatedSVD

In [ ]:
svd = TruncatedSVD(n_components = 99)
latent_matrix_2 = svd.fit_transform(user_movies_data)
latent_matrix_2_df = pd.DataFrame(latent_matrix_2, index = id_title['title'].tolist())

In [ ]:
latent_matrix_2_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98
Heat (1995),-1.144837e-30,-0.000000e+00,-0.000000e+00,-4.376124e-45,0.000000e+00,0.000000e+00,-4.030408e-30,0.000000e+00,0.000000e+00,-1.242398e-15,-5.723626e-30,2.246958e-30,1.502547e-31,-0.000000e+00,0.000000e+00,2.774935e-29,-4.534875e-16,6.353739e-51,0.000000e+00,-0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00,0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00,2.544747e-31,-0.000000e+00,-0.000000e+00,0.000000e+00,...,-0.000000e+00,-0.000000e+00,0.000000e+00,-3.400451e-30,-0.000000e+00,1.117739e-15,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00,-2.512764e-31,-0.000000e+00,-0.000000e+00,9.268339e-32,-0.000000e+00,-0.000000e+00,5.219338e-31,-1.681996e-31,1.806335e-16,-1.302943e-17,-1.087551e-31,0.000000e+00,-8.998958e-46,8.794192e-32,0.000000e+00,0.000000e+00,-1.646119e-31,-6.374151e-32,-0.000000e+00,1.270611e-16,4.861852e-17,-1.226652e-16,-2.600008e-22,-1.446049e-32,5.318227e-33,-3.450301e-17,1.871605e-17,7.342608e-34,-1.517392e-17
Powder (1995),2.114061e-16,-7.318604e-16,-9.123273e-16,2.834867e-15,-5.146477e-16,-7.106371e-16,-7.522263e-16,-9.197409e-16,-1.276453e-15,1.942890e-15,-3.414903e-16,3.732115e-16,-1.985358e-16,6.763826e-16,1.696077e-16,4.561602e-16,0.000000e+00,-1.814000e-16,1.541576e-16,2.895097e-16,-1.587499e-16,2.555176e-16,6.834691e-16,-6.229143e-16,-1.361608e-16,3.549108e-16,2.076457e-16,1.288859e-15,7.142184e-18,-4.457732e-16,6.236822e-16,-1.584815e-16,-2.963103e-16,-4.497201e-16,-3.230407e-16,1.690157e-16,1.767366e-16,-4.255657e-16,-4.710274e-16,2.475073e-18,...,3.444499e-16,-1.078867e-15,2.644092e-16,-6.080793e-16,3.719915e-16,-6.661338e-16,-3.226299e-16,-2.245898e-16,-2.726884e-16,-1.762721e-15,-7.027832e-18,4.611740e-16,-3.880123e-16,-2.450529e-17,2.732921e-16,-3.725172e-16,-3.232873e-16,-1.092245e-16,3.527742e-16,-1.734723e-16,-6.570265e-16,-6.841407e-16,3.424411e-01,5.940996e-01,1.152007e+00,4.239275e-01,9.163628e-03,8.756470e-01,-6.204679e-01,-6.876652e-01,-1.598465e-01,6.104817e-01,9.159340e-16,-2.503059e-16,-6.387648e-17,-9.728603e-17,-3.261280e-16,-6.279699e-16,5.517130e-18,2.081668e-17
Carrington (1995),-4.451593e-17,-1.641636e-15,-3.596347e-15,2.285506e-15,-5.006152e-15,-1.175274e-15,2.431827e-15,5.576412e-16,-4.145083e-15,-2.220446e-15,1.989504e-17,1.612136e-15,-4.640096e-15,4.962488e-15,1.427821e-15,-5.542658e-15,-2.123302e-15,-1.170542e+00,-3.496169e-01,2.112102e-01,-1.274897e+00,-8.637873e-01,-3.703964e-01,6.376974e-01,-3.922496e-01,-9.653719e-02,1.526967e+00,-2.690457e-01,-9.748079e-01,5.400751e-01,-5.227318e-01,-1.158587e+00,5.680853e-01,-2.125497e-01,-2.613045e-01,6.251211e-01,1.990730e+00,-5.077626e-01,-3.250816e-01,-7.861061e-01,...,-6.393548e-16,-1.644912e-16,-6.694656e-16,1.902188e-16,-8.310063e-16,9.992007e-16,-1.315667e-15,1.085003e-15,-2.558669e-16,7.149950e-17,-8.673617e-19,8.002978e-16,-1.008178e-15,-9.115952e-16,-4.633694e-16,-3.104620e-16,-9.689055e-16,-1.073443e-15,-2.885672e-16,3.469447e-16,3.512815e-16,3.363616e-16,-7.890281e-17,-1.204549e-16,4.928591e-17,-3.794708e-19,-2.468334e-16,-3.952674e-17,2.413950e-17,-2.202015e-16,-1.665335e-16,6.938894e-17,5.412337e-16,2.415419e-16,2.761969e-17,7.607083e-17,-1.387779e-17,-4.163336e-17,-6.574553e-18,6.938894e-18
Seven (a.k.a. Se7en) (1995),-5.572439e-16,-1.110415e-15,1.428602e-15,-2.101075e-15,7.946642e-16,-3.424327e-16,-3.866676e-16,2.015134e-16,1.327919e-15,2.775558e-16,2.725440e-16,-3.131146e-16,2.494949e-16,-2.166369e-16,-3.908941e-16,-3.342537e-17,6.245005e-16,-2.914335e-16,2.072995e-16,4.128642e-16,-3.157197e-16,-1.214306e-16,1.125836e-15,-3.226586e-16,2.827599e-16,-2.905662e-17,5.481726e-16,-1.882175e-16,-3.018419e-16,-6.817463e-16,4.198031e-16,-3.9

Hybrid

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def recommend_similar_movies(title, method = 'hybrid'):
    
    a_1 = np.array(latent_matrix_1_df.loc[title]).reshape(1, -1)
    a_2 = np.array(latent_matrix_2_df.loc[title]).reshape(1, -1)


# calculate the similartity of this movie with the others in the list
    score_content = cosine_similarity(latent_matrix_1_df, a_1).reshape(-1)
    score_collab = cosine_similarity(latent_matrix_2_df, a_2).reshape(-1)

# hybrid score: an average measure of both content and collaborative 
    hybrid_score = (score_content + score_collab)

# form a data frame of similar movies 
    dictDF = {'content': score_content, 'collab': score_collab, 'hybrid': hybrid_score}

    similar_movies = pd.DataFrame(dictDF, index = latent_matrix_2_df.index)

#sort it on the basis of either: content, collaborative or hybrid, here : content
    similar_movies.sort_values(method, ascending = False, inplace = True)
    print("Top 10 Movie Recommendations are: ")
    print(similar_movies.head(10))
    
    return None

In [ ]:
#Hybrid Method
recommend_similar_movies("Chinatown (1974)", 'hybrid')

Top 10 Movie Recommendations are: 
                                                     content  ...    hybrid
Chinatown (1974)                                    1.000000  ...  2.000000
Five Easy Pieces (1970)                             0.287531  ...  0.287531
Narc (2002)                                         0.086448  ...  0.086448
Others, The (2001)                                  0.069452  ...  0.069452
Heat (1995)                                         0.066395  ...  0.066395
Blade Runner (1982)                                 0.054180  ...  0.054180
Cell, The (2000)                                    0.045612  ...  0.045612
Lord of the Rings: The Fellowship of the Ring, ...  0.045595  ...  0.045595
Seven (a.k.a. Se7en) (1995)                         0.043218  ...  0.043218
Kill Bill: Vol. 2 (2004)                            0.042028  ...  0.042028

[10 rows x 3 columns]


Matrix Factorization Recommender

In [ ]:
class MF():

    # Initializing the user-movie rating matrix, no. of latent features, alpha and beta.
    def __init__(self, R, K, alpha, beta, iterations):
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    # Initializing user-feature and movie-feature matrix 
    def train(self):
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initializing the bias terms
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])

        # List of training samples
        self.samples = [
        (i, j, self.R[i, j])
        for i in range(self.num_users)
        for j in range(self.num_items)
        if self.R[i, j] > 0
        ]

        # Stochastic gradient descent for given number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            if (i+1) % 20 == 0:
                print("Iteration: %d ; error = %.4f" % (i+1, mse))

        return training_process

    # Computing total mean squared error
    def mse(self):
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    # Stochastic gradient descent to get optimized P and Q matrix
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_rating(i, j)
            e = (r - prediction)

            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    # Ratings for user i and moive j
    def get_rating(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    # Full user-movie rating matrix
    def full_matrix(self):
        return mf.b + mf.b_u[:,np.newaxis] + mf.b_i[np.newaxis:,] + mf.P.dot(mf.Q.T)

In [ ]:
R= np.array(rating_df1.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0))

In [ ]:
mf = MF(R, K=20, alpha=0.001, beta=0.01, iterations=100)
training_process = mf.train()

Iteration: 20 ; error = 10.4294
Iteration: 40 ; error = 9.9994
Iteration: 60 ; error = 9.5872
Iteration: 80 ; error = 9.1918
Iteration: 100 ; error = 8.8126


In [ ]:
mf.full_matrix()

array([[3.38671118, 3.32075389, 3.49210314, ..., 3.40453043, 3.44121054,
        3.38516619],
       [3.35969219, 3.23398978, 3.44967831, ..., 3.3359587 , 3.39563218,
        3.33447751],
       [3.41170276, 3.28625235, 3.48333799, ..., 3.38868195, 3.43984842,
        3.3967338 ],
       ...,
       [3.22583982, 3.11363395, 3.30218301, ..., 3.20952127, 3.23664895,
        3.2154354 ],
       [3.19948082, 3.11840533, 3.30188344, ..., 3.19977732, 3.25236428,
        3.20515592],
       [3.10512187, 3.03988087, 3.21279833, ..., 3.13860655, 3.18071139,
        3.13653333]])

In [ ]:
#user 99 for movie 2
mf.get_rating(99, 2)

3.2127983266352054

Turicreate

In [ ]:
import turicreate

In [ ]:
uploaded = files.upload()

Saving ratings_samp_te.csv to ratings_samp_te.csv
Saving ratings_samp_test.csv to ratings_samp_test.csv


In [ ]:
ratings_train = pd.read_csv("ratings_samp_te.csv")
ratings_test = pd.read_csv("ratings_samp_test.csv")

In [ ]:
train_data  = turicreate.SFrame(ratings_train)
test_data  = turicreate.SFrame(ratings_test)

Popularity

In [ ]:
pop = turicreate.popularity_recommender.create(train_data, user_id='userId', item_id="movieId", target="rating")

Warning: Ignoring columns timestamp;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 60001 observations with 37764 users and 7144 items.

Data prepared in: 0.077246s

60001 observations to process; with 7144 unique items.

In [ ]:
pop.recommend(users = [1,2,3], k=5).print_rows(num_rows = 15)

+--------+---------+-------+------+
| userId | movieId | score | rank |
+--------+---------+-------+------+
|   1    |   3622  |  5.0  |  1   |
|   1    |   3963  |  5.0  |  2   |
|   1    |   297   |  5.0  |  3   |
|   1    |  62792  |  5.0  |  4   |
|   1    |  114060 |  5.0  |  5   |
|   2    |   3622  |  5.0  |  1   |
|   2    |   3963  |  5.0  |  2   |
|   2    |   297   |  5.0  |  3   |
|   2    |  62792  |  5.0  |  4   |
|   2    |  114060 |  5.0  |  5   |
|   3    |   3622  |  5.0  |  1   |
|   3    |   3963  |  5.0  |  2   |
|   3    |   297   |  5.0  |  3   |
|   3    |  62792  |  5.0  |  4   |
|   3    |  114060 |  5.0  |  5   |
+--------+---------+-------+------+
[15 rows x 4 columns]



Surprise

In [ ]:
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split

In [ ]:
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(rating_df1[['userId', 'movieId', 'rating']], reader)

In [ ]:
trainset, testset = train_test_split(data, test_size = 0.25)

In [ ]:
svd = SVD()

In [ ]:
svd.fit(trainset)

In [ ]:
Mapping_file = dict(zip(movie_df['title'].tolist(), movie_df['movieId'].tolist()))

In [ ]:
def pred_user_rating(ui):
    if ui in rating_df1.userId.unique():
        ui_list = rating_df1[rating_df1.userId == ui].movieId.tolist()
        d = {k: v for k,v in Mapping_file.items() if not v in ui_list}        
        predictedL = []
        for i, j in d.items():     
            predicted = svd.predict(ui, j)
            predictedL.append((i, predicted[3])) 
        pdf = pd.DataFrame(predictedL, columns = ['movies', 'ratings'])
        pdf.sort_values('ratings', ascending=False, inplace=True)  
        pdf.set_index('movies', inplace=True)    
        return pdf.head(10)        
    else:
        print("User Id does not exist in the list!")
        return None

In [ ]:
pred_user_rating(4829)

,ratings
movies,
Secretary (2002),3.657892
"Bank Job, The (2008)",3.644762
Narc (2002),3.629050
"Adventures of Priscilla, Queen of the Desert, The (1994)",3.589467
Snatch (2000),3.580675
Casino Royale (2006),3.574980
Trainspotting (1996),3.566250
"Impostors, The (1998)",3.564256
Beverly Hills Cop (1984),3.547632


In [ ]:
pred_user_rating(24)

User Id does not exist in the list!
